# Fine-tuning with QLoRA on Colab Pro GPU

Notebook này đã được tối ưu để sử dụng GPU trên Google Colab Pro (A100). 

## Kiểm tra GPU và Cấu hình

Đầu tiên, chúng ta sẽ kiểm tra loại GPU được cấp phát và cấu hình các thông số cho phù hợp.

# Fine-tuning with QLoRA (Colab Version)

This notebook demonstrates how to fine-tune a language model using QLoRA (Quantized Low-Rank Adaptation) for text summarization tasks in Google Colab.

## 1. Setup Environment

First, we'll mount Google Drive and install required packages.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set up project directory
import os

# Change this to your Google Drive project path
PROJECT_DIR = "/content/drive/MyDrive/text_summarization_project"
DATA_DIR = os.path.join(PROJECT_DIR, "data")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed")
MODEL_DIR = os.path.join(PROJECT_DIR, "models")
QLORA_MODEL_DIR = os.path.join(MODEL_DIR, "finetuned_qlora")

# Create necessary directories
os.makedirs(QLORA_MODEL_DIR, exist_ok=True)

# Install required packages
!pip install -q transformers datasets accelerate bitsandbytes peft trl
!pip install -q torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2
!pip install -q evaluate rouge-score numpy pandas matplotlib seaborn

## 2. Load and Prepare Data

Load the preprocessed data from the previous notebook.

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load the preprocessed data
train_df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "train.csv"))
val_df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "validation.csv"))
test_df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "test.csv"))

# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Create dataset dictionary
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

## 3. Initialize Model and Tokenizer

Load the base model and configure it for QLoRA fine-tuning.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
import torch

# Model configuration
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

## 4. Configure QLoRA

Set up the QLoRA configuration for fine-tuning.

In [ ]:
from peft import LoraConfig

# LoRA configuration
lora_config = LoraConfig(
    r=8,                     # Rank
    lora_alpha=32,          # Alpha scaling
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA config to the model
model = get_peft_model(model, lora_config)

## 5. Training Configuration

Set up the training arguments and prepare for training.

In [ ]:
from transformers import TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir=QLORA_MODEL_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_8bit",
    logging_steps=10,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none"
)

## 6. Start Training

Initialize the trainer and start the fine-tuning process.

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# Start training
trainer.train()

## 7. Save and Upload Model

Save the trained model and upload it back to Google Drive.

In [ ]:
# Save the model
trainer.save_model(QLORA_MODEL_DIR)

# Upload the model back to Drive
!zip -r /content/qlora_model.zip $QLORA_MODEL_DIR
from google.colab import files
files.download('/content/qlora_model.zip')

In [ ]:
# Kiểm tra GPU
!nvidia-smi

import torch
print("\nGPU có sẵn:", torch.cuda.is_available())
print("Tên GPU:", torch.cuda.get_device_name(0))
print("Số lượng GPU:", torch.cuda.device_count())
print("CUDA version:", torch.version.cuda)

# Kiểm tra memory
!free -h

## Tối ưu hóa cho GPU

Cấu hình các thông số để tận dụng tối đa GPU của Colab Pro:

In [ ]:
# Cấu hình PyTorch để tối ưu hiệu năng
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Cấu hình cho mixed precision training
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(
    gradient_accumulation_steps=4,
    mixed_precision="bf16",  # Sử dụng bfloat16 cho A100
    log_with="tensorboard",
    fsdp_plugin=fsdp_plugin
)

## Cấu hình QLoRA cho GPU Colab Pro

Điều chỉnh các tham số của QLoRA để tận dụng tối đa VRAM của GPU:

In [ ]:
# Cấu hình QLoRA được tối ưu cho GPU A100
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,  # Sử dụng bfloat16 cho A100
    bnb_4bit_use_double_quant=True,  # Bật double quantization để tiết kiệm memory
)

# Load model với cấu hình QLoRA tối ưu
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,  # Sử dụng bfloat16
    use_cache=False
)

# Cấu hình LoRA với các tham số tối ưu cho GPU mạnh
lora_config = LoraConfig(
    r=16,  # Tăng rank lên vì có nhiều VRAM
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Thêm k_proj và o_proj
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

## Cấu hình Training Arguments

Điều chỉnh các tham số huấn luyện để tận dụng tối đa GPU:

In [ ]:
# Cấu hình training được tối ưu cho GPU A100
training_args = TrainingArguments(
    output_dir=QLORA_MODEL_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=8,  # Tăng batch size vì có nhiều VRAM
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Giảm gradient accumulation vì batch size đã lớn
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    learning_rate=2e-4,
    bf16=True,  # Sử dụng bfloat16 cho A100
    optim="adamw_torch_fused",  # Sử dụng fused optimizer để tăng tốc
    logging_steps=10,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    gradient_checkpointing=True,  # Bật gradient checkpointing để tiết kiệm memory
    group_by_length=True,  # Nhóm các sequences có độ dài tương tự để tối ưu memory
)

# Enable tensor cores for faster training
torch.set_float32_matmul_precision('high')

In [ ]:
# Hàm tính thời gian còn lại
import time
from datetime import timedelta

class TrainingMonitor:
    def __init__(self, total_steps):
        self.total_steps = total_steps
        self.start_time = None
        self.step_times = []
        
    def start(self):
        self.start_time = time.time()
        
    def update(self, current_step):
        if len(self.step_times) == 0:
            self.step_times.append(time.time() - self.start_time)
        else:
            self.step_times.append(time.time() - (self.start_time + sum(self.step_times)))
        
        # Tính thời gian trung bình mỗi step
        avg_step_time = sum(self.step_times[-50:]) / min(len(self.step_times), 50)
        
        # Tính thời gian còn lại
        steps_remaining = self.total_steps - current_step
        time_remaining = steps_remaining * avg_step_time
        
        # Tính tốc độ training
        steps_per_second = 1 / avg_step_time
        
        return {
            'avg_step_time': avg_step_time,
            'time_remaining': str(timedelta(seconds=int(time_remaining))),
            'steps_per_second': steps_per_second,
            'progress': f"{current_step}/{self.total_steps} ({(current_step/self.total_steps*100):.1f}%)"
        }

# Khởi tạo training monitor
monitor = None  # Sẽ được khởi tạo sau khi biết tổng số steps

## Rút gọn Dataset

Để giảm thời gian training, chúng ta sẽ sử dụng một tập con của dataset.

In [ ]:
# Rút gọn dataset để test nhanh
MAX_TRAIN_SAMPLES = 5000  # Số lượng mẫu train
MAX_VAL_SAMPLES = 1000    # Số lượng mẫu validation

print(f"Kích thước dataset ban đầu:")
print(f"- Train: {len(dataset['train'])} mẫu")
print(f"- Validation: {len(dataset['validation'])} mẫu")

# Rút gọn dataset
dataset["train"] = dataset["train"].select(range(min(MAX_TRAIN_SAMPLES, len(dataset["train"]))))
dataset["validation"] = dataset["validation"].select(range(min(MAX_VAL_SAMPLES, len(dataset["validation"]))))

print(f"\nKích thước dataset sau khi rút gọn:")
print(f"- Train: {len(dataset['train'])} mẫu")
print(f"- Validation: {len(dataset['validation'])} mẫu")

In [ ]:
# Cập nhật training arguments cho dataset nhỏ hơn
training_args = TrainingArguments(
    output_dir=QLORA_MODEL_DIR,
    num_train_epochs=1,  # Giảm số epochs
    per_device_train_batch_size=16,  # Tăng batch size
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,  # Giảm gradient accumulation vì dataset nhỏ
    evaluation_strategy="steps",
    eval_steps=50,  # Đánh giá thường xuyên hơn
    save_strategy="steps",
    save_steps=50,
    learning_rate=2e-4,
    bf16=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    gradient_checkpointing=True,
    group_by_length=True,
)

In [ ]:
# Khởi tạo trainer với callback để theo dõi thời gian
from transformers import TrainerCallback

class TimeMonitorCallback(TrainerCallback):
    def __init__(self, monitor):
        self.monitor = monitor
    
    def on_train_begin(self, args, state, control, **kwargs):
        self.monitor.start()
    
    def on_step_end(self, args, state, control, **kwargs):
        stats = self.monitor.update(state.global_step)
        print(f"\rTiến độ: {stats['progress']} | "
              f"Thời gian còn lại: {stats['time_remaining']} | "
              f"Tốc độ: {stats['steps_per_second']:.2f} steps/s", end="")

# Tính tổng số steps
total_steps = int(len(dataset["train"]) / training_args.per_device_train_batch_size / training_args.gradient_accumulation_steps * training_args.num_train_epochs)
monitor = TrainingMonitor(total_steps)

# Khởi tạo trainer với callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[TimeMonitorCallback(monitor)]
)

print(f"Tổng số steps: {total_steps}")
print(f"Ước tính thời gian training (dựa trên 1.73s/step): {str(timedelta(seconds=int(total_steps * 1.73)))}")
print("\nBắt đầu training...")

In [ ]:
# Bắt đầu training
trainer.train()